<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [4]:
from scipy.sparse import lil_matrix
import edward as ed
from edward.models import Beta
import tensorflow as tf
import numpy as np

In [132]:
with open('kronecker-core-periphery-n1024-h10-r0_01-0_25-1000-cascades.txt','r') as f:
    
    # Store number of nodes
    numNodes = -1
    while True:
        if f.readline() == "\n":
            break
        numNodes+=1

    # Collect cascades into list
    v = []
    for line in f.readlines():
        v.append([float(l) for l in line.rstrip('\n').split(",")])

In [297]:
np_cascades = np.zeros((len(v),numNodes))
for row, cascade in enumerate(v):  
    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]

    for col in range(len(c_nodes)):
        np_cascades[row][c_nodes[col]]=c_times[col]

dtype('float64')

In [201]:
def infectedCascade(cascade, start,N=numNodes, T=10):
    A_inf = lil_matrix((cascade.shape[0],cascade.shape[0]))

    infected = np.nonzero(cascade)[0]
    c_nodes= cascade.argsort()[-len(infected):]

    for i in range(len(c_nodes)):
        for j in range(i):
            if cascade[j] < T:
                A_inf[(c_nodes[i],c_nodes[j])]=cascade[i]-cascade[j]
    
    return A_inf

In [249]:
def uninfectedCascade(cascade,start,N=numNodes,T=10):
    nodes = {s for s in range(N)}
    A_uninf = lil_matrix((N,N))

    infected = np.nonzero(cascade)[0]
    c_nodes= cascade.argsort()[-len(infected):]

    for i in range(len(c_nodes)):
        for j in (nodes-set(c_nodes)):
            A_uninf[c_nodes[i],j]=T-cascade[i]

    return A_uninf

In [305]:
numNodes
np_cascades.shape


(1001, 1023)

In [304]:
A = Beta(tf.ones([numNodes,numNodes]),tf.ones([numNodes,numNodes]))
cascade_ph = Cascade(A)

sess.run(cascade_ph._log_prob(np_cascades[0]))

# dir(cascade_ph)

AttributeError: 'Cascade' object has no attribute '_params'

In [285]:
np_cascades.shape

(1001, 1023)

In [286]:
data = {cascade_ph: np_cascades[0]}

In [287]:
inference = ed.MAP([A], data)

/Users/brianallen131/src/edward/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


In [289]:
sess = ed.get_session()
tf.global_variables_initializer().run()

inference.run(n_iter=2500)

TypeError: __init__() got an unexpected keyword argument 'dtype'

In [294]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.framework import tensor_shape

from edward.models.random_variable import RandomVariable
from tensorflow.contrib.distributions import Distribution

try:
    from tensorflow.contrib.distributions import FULLY_REPARAMETERIZED
except Exception as e:
    raise ImportError("{0}. Your TensorFlow version is not supported.".format(e))

class Cascade(RandomVariable, Distribution):
    def __init__(self,
                params,
                validate_args=False,
                allow_nan_stats=True,
                name="Cascade"):
        """Initialize a `Cascade` random variable.
        Args:
            params: tf.Tensor.
            Collection of samples. Its outer (left-most) dimension
            determines the number of samples.
        """
        parameters = locals()
        with tf.name_scope(name, values=[params]):
            with tf.control_dependencies([]):
                self._params = tf.identity(params, name="params")
                try:
                    self._n = tf.shape(self._params)[0]
                except ValueError:  # scalar params
                    self._n = tf.constant(1)
                    
        super(Cascade, self).__init__(
            dtype=self._params.dtype,
            reparameterization_type=FULLY_REPARAMETERIZED,
            validate_args=validate_args,
            allow_nan_stats=allow_nan_stats,
            parameters=parameters,
            graph_parents=[self._params, self._n],
            name=name)
    
    @staticmethod
    def _param_shapes(sample_shape):
        return {"params": tf.convert_to_tensor(sample_shape, dtype=tf.int32)}

    @property
    def params(self):
        """Distribution parameter."""
        return self._params

    @property
    def n(self):
        """Number of samples."""
        return self._n

    def _batch_shape_tensor(self):
        return array_ops.shape(self.params)

    def _batch_shape(self):
        return self.params.shape[1:]

    def _event_shape_tensor(self):
        return tf.shape(self.params)[1:]

    def _event_shape(self):
        return tensor_shape.scalar()


    
    def _log_prob(self, value):
        # TODO
        alpha_tensor = tf.convert_to_tensor(tf.cast(self.params,dtype=tf.float64),dtype=tf.float64)
        
        I = tf.convert_to_tensor(infectedCascade(value,0,N=alpha_tensor.shape[0]).todense(),dtype = tf.float64)
        U = tf.convert_to_tensor(uninfectedCascade(value,0,N=alpha_tensor.shape[0]).todense(),dtype = tf.float64)


        psi_1 = -tf.reduce_sum(tf.multiply(alpha_tensor,I))
        psi_2 = -tf.reduce_sum(tf.multiply(tf.transpose(alpha_tensor),U))
        
        zero_pos = tf.add(tf.multiply(tf.sign(I),-1),tf.ones((self.params.shape[0],self.params.shape[0]),dtype=tf.float64))
        psi_3 = tf.reduce_sum(tf.log(tf.add(zero_pos,tf.multiply(tf.transpose(alpha_tensor),tf.sign(I)))))

        return psi_1 + psi_2 + psi_3

    def _sample_n(self, n, seed=None):
        input_tensor = self.params
        if len(input_tensor.shape) == 0:
            input_tensor = tf.expand_dims(input_tensor, 0)
            multiples = tf.concat([tf.expand_dims(n, 0), [1] * len(self.event_shape)], 0)
            return tf.tile(input_tensor, multiples)
        else:
            probs = tf.ones([self.n]) / tf.cast(self.n, dtype=tf.float32)
            cat = tf.contrib.distributions.Categorical(probs)
            indices = cat._sample_n(n, seed)
            tensor = tf.gather(input_tensor, indices)
            return tensor

In [295]:
alpha_test = np.random.uniform(0,1,[1023,1023])
# print(v[0])
x = Cascade(alpha_test)
sess = tf.InteractiveSession()
sess.run(x.sample())


AttributeError: 'Cascade' object has no attribute '_params'

In [229]:
alpha_test = np.ones((3,3),dtype=np.float64)/2

testcascade = Cascade(alpha_test)

sess = tf.InteractiveSession()
test_list = [1., 0., 0., 1.5]
sess.run(testcascade._log_prob(test_list))

AttributeError: 'list' object has no attribute 'shape'

In [237]:
sess.run(cascade_ph._log_prob(v[0]))

-24725.090304108082

In [183]:
from edward.models import Dirichlet, Beta, Multinomial, Bernoulli
K=100
N=50

gamma = Dirichlet(concentration=tf.ones([K]))
Pi = Beta(concentration0=tf.ones([K, K]), concentration1=tf.ones([K, K]))
Z = Multinomial(total_count=1.0, probs=gamma, sample_shape=N)
X = Bernoulli(probs=tf.matmul(Z, tf.matmul(Pi, tf.transpose(Z))))

gamma.event_shape

TensorShape([Dimension(100)])